### Logic to Karnaugh map

[Wiki: Karnaugh map](https://en.wikipedia.org/wiki/Karnaugh_map)
![](https://upload.wikimedia.org/wikipedia/commons/thumb/0/02/K-map_6%2C8%2C9%2C10%2C11%2C12%2C13%2C14_anti-race.svg/440px-K-map_6%2C8%2C9%2C10%2C11%2C12%2C13%2C14_anti-race.svg.png)

# 导入

In [1]:
from sympy.combinatorics.graycode import GrayCode
from itertools import permutations
import numpy as np

In [2]:
def ReverseBinCode(l):
    # Generate reversed binary code for pyphi
    maxIndex = 2 ** l - 1
    return [np.binary_repr(maxIndex - index, width=l) for index in range(2 ** l)]

# 函数 Functions

## 重标记 Re-labeling

In [3]:
def code2str(codeList):
    s = ''
    for i in codeList:
        s += str(i)
    return s

def relabel(m0, order):
    length = len(m0[0])
    bcode = ReverseBinCode(length)
    reorderRule = {}
    for originLabel, newLabel in zip(bcode, order):
        reorderRule[originLabel] = newLabel
    #print(reorderRule)
    rule = {}
    # 初始化rule
    for currentState, nextState in zip(bcode, m0):
        rule[code2str(currentState)] = nextState
    # 更新顺序
    for currentState, nextState in zip(bcode, m0):
        #print(reorderRule[code2str(currentState)], 'to', str(nextState))
        rule[code2str(reorderRule[code2str(currentState)])] = reorderRule[code2str(nextState)]
    
    new_m = [rule[startState] for startState in ReverseBinCode(length)]
    
    return np.array(new_m)

def AllLabelConfigurations(mat):
    # 返回所有可能的重标记方法
    _check = {}
    all_elements = [
    [1,1,1],
    [1,1,0],
    [1,0,1],
    [1,0,0],
    [0,1,1],
    [0,1,0],
    [0,0,1],
    [0,0,0]
    ]
    for order in permutations(all_elements):
        _relabel = relabel(mat, list(order))
        _check[str(_relabel)] = _relabel
    return list(_check.values())

## Karnaugh Map & Karnaugh-Ising Energy

### 输入`pyphi`中状态转移矩阵，给出各个变量的卡诺图

In [4]:
# 输入状态转移列表，给出各个量的卡诺图矩阵

def MapToKarnaughMap(_map):
    # Turn map to MapToKarnaugh Maps
    # Generate ReverseBinCode
    index = ReverseBinCode(len(_map[0]))
    #index = np.array([[int(char) for char in strcode] for strcode in index])
    
    dim1 = (int(len(_map[0]) / 2))
    dim2 = (len(_map[0]) - int(len(_map[0]) / 2))
    #print('dim =', dim1, 'x', dim2)
    
    gc1 = GrayCode(dim1)
    gc2 = GrayCode(dim2)
    gd1 = dict(zip(list(gc1.generate_gray()), range(2 ** dim1)))
    gd2 = dict(zip(list(gc2.generate_gray()), range(2 ** dim2)))
    #print(gd1)
    #print(gd2)
    
    _map = np.array(_map).T
    KarnaughMaps = []
    for targetStates in _map:
        #print('-------')
        Karnaugh = np.array(([[0] * (2 ** dim1)]) * (2 ** dim2)).T
        #print(Karnaugh)
        for state, mcode in zip(index, targetStates):
            #print(state, '->', mcode)
            #print('c1=',state[0:dim1])
            #print('c2=',state[dim1:])
            Karnaugh[gd1[state[0:dim1]], gd2[state[dim1:]]] = mcode
        KarnaughMaps += [Karnaugh]
        #print(Karnaugh)
    
    return np.array(KarnaughMaps)

### 输入矩阵，给出Ising模型能量

In [5]:
def IsingEnergy(mat, energy={0:-1, 1:1}):
    # 给出mat的Ising能量
    # Return Ising energy of mat
    mat = np.array(mat)
    # If mat is a list of mat, then return all energy
    if len(mat.shape) == 3:
        return np.array([IsingEnergy(_mat, energy) for _mat in mat])
    # Otherwise return this energy
    x, y = mat.shape
    total_E = 0
    for i in range(x):
        for j in range(y):
            total_E += energy[mat[i][j]] * energy[mat[(i - 1) % x][j]]
            total_E += energy[mat[i][j]] * energy[mat[(i + 1) % x][j]]
            total_E += energy[mat[i][j]] * energy[mat[i][(j - 1) % y]]
            total_E += energy[mat[i][j]] * energy[mat[i][(j + 1) % y]]
    return -total_E

In [6]:
m = [
    [0,0,0],
    [0,0,1],
    [0,1,0],
    [0,1,1],
    [1,0,0],
    [1,0,1],
    [1,1,0],
    [1,1,1]
    ]
MapToKarnaughMap(m)

array([[[1, 1, 1, 1],
        [0, 0, 0, 0]],

       [[1, 1, 0, 0],
        [1, 1, 0, 0]],

       [[1, 0, 0, 1],
        [1, 0, 0, 1]]])

### 计算不同排序的Ising能量

In [54]:
m = [
    [0,0,0],
    [0,0,1],
    [0,1,0],
    [0,1,1],
    [1,0,0],
    [1,0,1],
    [1,1,0],
    [1,1,1]
    ]
energyList = []
maxlen = 50

testorders=permutations(m)

for i, order in zip(range(maxlen), testorders):
    current_m = relabel(m, list(order))
    energy = [IsingEnergy(kmap) for kmap in MapToKarnaughMap(current_m)]
    energyList += energy

print(energyList)

[0, -16, -16, 0, -16, 32, 0, -16, -16, 0, 8, 8, 0, 8, 8, 0, 32, -16, 0, -16, 32, 0, -16, -16, 0, 8, 8, 0, 0, -16, 0, 16, 16, 0, 8, 8, 0, 8, 8, 0, 16, 16, 0, 32, -16, 0, 8, 8, 0, -16, -16, 0, -16, 0, 0, 0, -16, 0, 8, 8, 0, 8, 8, 0, -16, -16, 0, -16, 0, 0, -16, -16, 0, -16, -16, 0, -16, 32, 0, -16, -16, 0, 8, 8, 0, 8, 8, 0, 32, -16, -16, 0, -16, -16, 0, 32, 0, 0, 0, 8, 0, 8, 0, 16, 16, 8, 16, -16, 0, 0, 0, 0, 16, 16, -16, -16, 0, 8, -16, 16, -16, 32, 0, 8, 8, 0, 8, 0, 8, 8, 16, -16, 8, -16, 16, 32, -16, -16, 8, 8, 0, 32, -16, -16, 0, -16, 32, 0, -16, -16]


## NN Energy / Cost

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

In [465]:
class net(nn.Module):
    # 单个神经网络
    def __init__(self, size_vis, size_hidden):
        super(net, self).__init__()
        self.fcn_v = nn.Linear(size_vis, size_vis)
        self.fcn_v2h = nn.Linear(size_vis, size_hidden)
        self.fcn_h2v = nn.Linear(size_hidden, size_vis)
        
    def forward(self, x):
        h = torch.sigmoid(self.fcn_v2h(x))
        xp = torch.sigmoid(self.fcn_v(x))
        
        xout = torch.sigmoid(self.fcn_h2v(h) + self.fcn_v(xp))
        
        return xout
    
class multinet(nn.Module):
    # 多个并行的神经网络，用以加速训练
    def __init__(self, size_vis, size_hidden, total_num = 1):
        # size_vis: number of visable nodes
        # size_hidden: number of hidden nodes
        # total_num: number of parallel networks
        super(multinet, self).__init__()
        self.fcn_v2h = nn.ModuleList([nn.Linear(size_vis, size_hidden, bias=False) for i in range(total_num)])
        self.fcn_h2v = nn.ModuleList([nn.Linear(size_hidden, size_vis, bias=False) for i in range(total_num)])
        self.total_n = total_num
    
    def forward_one(self, x):
        h = [torch.sigmoid(f1(a_input)) for f1, a_input in zip(self.fcn_v2h, x)]
        xout = torch.stack([torch.sigmoid(f2(a_input)) for f2, a_input in zip(self.fcn_h2v, h)])
        return xout
    
    def forward(self, x):
        xout = torch.stack([self.forward_one(ax) for ax in x])
        return xout

In [408]:
def BinCodeList(length):
    # 给出长度为length的倒序01序列
    s = ReverseBinCode(length)

    return [[int(char) for char in ss] for ss in s]

class graphData(data.Dataset):
    #输入转移图，输出训练集（输入、输出）
    def __init__(self, _m):
        super(graphData, self).__init__()
        self.inputs = BinCodeList(len(m[0]))
        self.m = _m
        
    def __len__(self):
        return len(self.inputs)
    def __getitem__(self, index):
        return torch.Tensor(self.inputs[index]), torch.Tensor(self.m[index])
    
class multiGraphData(data.Dataset):
    #输入转移图，输出训练集（输入、输出）
    #输出部分扩增，以便并行计算
    def __init__(self, _m, total_num=1):
        super(multiGraphData, self).__init__()
        self.inputs = BinCodeList(len(m[0]))
        self.inputs = torch.Tensor([[am] * total_num for am in self.inputs])
        
        self.m = _m
        self.m = torch.Tensor([[am] * total_num for am in self.m])
        
    def __len__(self):
        return len(self.inputs)
    def __getitem__(self, index):
        return self.inputs[index], self.m[index]

In [467]:
#同时训练10个平行的网络

IITnets = multinet(3, 4, total_num=3)

print(IITnets.forward(torch.Tensor([[[1,1,1],[1,1,1],[1,1,1]]])))

tensor([[[0.5614, 0.5758, 0.4821],
         [0.4393, 0.4943, 0.5388],
         [0.5340, 0.5007, 0.5857]]], grad_fn=<StackBackward>)


In [340]:
#Test

m = [
    [0,0,0],
    [0,0,1],
    [0,1,0],
    [0,1,1],
    [1,0,0],
    [1,0,1],
    [1,1,0],
    [1,1,1]
    ]

dataset = multiGraphData(m, total_num=3)
setLoader = data.DataLoader(dataset, batch_size = 1, shuffle=True)

### 测试单个神经网络的训练

In [110]:
IITnet = net(3, 4)



dataset = graphData(m)
setLoader = data.DataLoader(dataset, batch_size = 1, shuffle=True)
optimizer = optim.SGD(IITnet.parameters(), lr = 0.01, momentum=0.9, weight_decay=0.0005)

loss_fn = torch.nn.MSELoss()

for epco in range(4000):
    for inputdata, target in setLoader:
        optimizer.zero_grad()
        #print(inputdata)
        #print(target)
        #inputdata = torch.Tensor(inputdata)
        output = IITnet.forward(inputdata)
        loss = loss_fn(output, torch.Tensor(target))
        loss.backward()
        optimizer.step()
    if epco % 400 == 0:
        print('epco', epco, 'loss:',loss.item())

epco 0 loss: 0.2835400700569153
epco 400 loss: 0.12037954479455948
epco 800 loss: 0.12377927452325821
epco 1200 loss: 0.07433902472257614
epco 1600 loss: 0.0601571649312973
epco 2000 loss: 0.0337505042552948
epco 2400 loss: 0.022427517920732498
epco 2800 loss: 0.035545188933610916
epco 3200 loss: 0.02056305482983589
epco 3600 loss: 0.02959204837679863


### 测试并行神经网络的训练

In [350]:
IITNets.forward(torch.Tensor([[1,1,1]]))

tensor([[[0.3830, 0.4067, 0.4171]],

        [[0.5226, 0.5306, 0.4608]],

        [[0.5504, 0.3943, 0.5537]],

        [[0.5637, 0.6044, 0.4760]],

        [[0.4598, 0.5796, 0.5974]]], grad_fn=<StackBackward>)

In [468]:
num = 4
IITNets = multinet(3,4,total_num=num)

dataset = multiGraphData(m, total_num=num)
setLoader = data.DataLoader(dataset, batch_size = 2, shuffle=True)

for x,y in setLoader:
    print('input:')
    print(x)
    print('forward:')
    print(IITNets.forward(x))
    print('answer:')
    print(y)

input:
tensor([[[0., 1., 1.],
         [0., 1., 1.],
         [0., 1., 1.],
         [0., 1., 1.]],

        [[1., 0., 1.],
         [1., 0., 1.],
         [1., 0., 1.],
         [1., 0., 1.]]])
forward:
tensor([[[0.6332, 0.5444, 0.5405],
         [0.6064, 0.4733, 0.3953],
         [0.3821, 0.5291, 0.5728],
         [0.6127, 0.4843, 0.3192]],

        [[0.6307, 0.5417, 0.5359],
         [0.6214, 0.4574, 0.3728],
         [0.3970, 0.5148, 0.5679],
         [0.5909, 0.4847, 0.3370]]], grad_fn=<StackBackward>)
answer:
tensor([[[1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.]],

        [[0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.]]])
input:
tensor([[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])
forward:
tensor([[[0.6414, 0.5389, 0.5464],
         [0.6133, 0.4586, 0.3893],
         [0.3773,

In [470]:
import time

start = time.time()

num = 5
IITNets = multinet(3,4,total_num=num)

dataset = multiGraphData(m, total_num=num)
setLoader = data.DataLoader(dataset, batch_size = 1, shuffle=True)
optimizer = optim.SGD(IITNets.parameters(), lr = 0.01, momentum=0.9, weight_decay=0.0005)

loss_fn = torch.nn.MSELoss()

for epco in range(2000):
    for inputdata, target in setLoader:
        optimizer.zero_grad()
        #print(inputdata)
        #print(target)
        #inputdata = torch.Tensor(inputdata)
        output = IITNets.forward(inputdata)
        loss = loss_fn(output, torch.Tensor(target))
        loss.backward()
        optimizer.step()
    if epco % 400 == 0:
        print('epco', epco, 'loss:',loss.item())
        
end = time.time()
print('total time cost:', end-start, 's')

epco 0 loss: 0.22591792047023773
epco 400 loss: 0.24268877506256104
epco 800 loss: 0.22384046018123627
epco 1200 loss: 0.2100454568862915
epco 1600 loss: 0.19776733219623566
total time cost: 10.187128782272339 s


In [ ]:
{{1, 3.9}, {5, 10.03}, {10, 17.07}, {20, 31.1},{40,58.9}}

In [447]:
IITNets.forward(torch.Tensor([[[0,1,1]]*5]))

tensor([[[0.8276, 0.3467, 0.2980],
         [0.7821, 0.3365, 0.3227],
         [0.7472, 0.2479, 0.3422],
         [0.8334, 0.3096, 0.4429],
         [0.7890, 0.3002, 0.3009]]], grad_fn=<StackBackward>)

In [155]:
def trainNet(_m, batchsize = 1, epco = 1400, print_loss=False, learning_rate = 0.002):
    _IITnet = net(3, 4)

    dataset = graphData(_m)
    setLoader = data.DataLoader(dataset, batch_size = batchsize, shuffle=True)
    optimizer = optim.SGD(_IITnet.parameters(), lr = learning_rate, momentum=0.9, weight_decay=1e-5)

    loss_fn = torch.nn.MSELoss()

    for epco in range(1400):
        for inputdata, target in setLoader:
            optimizer.zero_grad()
            output = _IITnet.forward(torch.Tensor(inputdata))
            loss = loss_fn(output, torch.Tensor(target))
            loss.backward()
            optimizer.step()
        #if epco % 400 == 0:
            #print('epco', epco, 'loss:',loss.item())
    if print_loss:
        print('loss:', loss.item())
    return _IITnet

In [159]:
import time

start = time.time()

trainNet(m, batchsize = 2, epco = 10000, print_loss=True, learning_rate=0.03)

end = time.time()
print('time cost:', end-start, 's')

loss: 0.04277151823043823
time cost: 1.9210000038146973 s


In [104]:
testorders=permutations(m)

energylist = []

for i,ms in zip(range(5),testorders):
    energylist += [ms, [NetEnergy(trainNet(ms)) for j in range(3)]]

print(energylist)

[([0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]), [113.74878692626953, 112.23719024658203, 112.02633666992188], ([0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 1], [1, 1, 0]), [99.32850646972656, 101.95207214355469, 92.23348999023438], ([0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 1, 0], [1, 0, 1], [1, 1, 1]), [109.4934310913086, 68.52285766601562, 69.92340850830078], ([0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 1, 0], [1, 1, 1], [1, 0, 1]), [98.31291198730469, 112.57115936279297, 122.3484115600586], ([0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 1, 1], [1, 0, 1], [1, 1, 0]), [103.64986419677734, 101.21601867675781, 93.5998306274414]]


In [16]:
NetEnergy(IITnet)

114.14593505859375

In [32]:
def NetEnergy(net):
    totalEnergy = 0.0
    for para in net.parameters():
        flat_para = para.view(para.numel())
        totalEnergy += sum(flat_para * flat_para)
    
    return totalEnergy.item()

In [105]:
len(list(testorders))

40315

In [16]:
m

[[0, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 1],
 [1, 0, 0],
 [1, 0, 1],
 [1, 1, 0],
 [1, 1, 1]]

In [18]:
relabel(m,[[0, 0, 0], [0, 0, 1], [1, 0, 1], [1, 0, 0], [1, 1, 0], [1, 1, 1], [1, 1, 1], [1, 1, 0]])

array([[0, 0, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 0],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 1],
       [1, 1, 0]])

In [19]:
len(AllLabelConfigurations(m))

105

In [14]:
all_labels = AllLabelConfigurations(m)

In [37]:
NetEnergy(trainNet(all_labels[0].tolist(), batchsize = 8))

81.47225952148438

In [31]:
energys = []

# expect time: 16min
start = time.time()
for i, graph in zip(range(5), all_labels):
    energys += [[NetEnergy(trainNet(graph.tolist())) for j in range(40)]]
    
end = time.time()
print(energys)
print('time cost:', end-start)

[[111.5406494140625, 109.67989349365234, 110.9552993774414, 108.75184631347656, 114.32801055908203, 112.37532043457031, 112.61180877685547, 109.82459259033203, 111.62741088867188, 113.04523468017578, 119.81019592285156, 112.51264953613281, 115.0881118774414, 109.69331359863281, 110.81085968017578, 110.8729248046875, 110.69241333007812, 107.08590698242188, 113.44341278076172, 114.39774322509766, 108.85160064697266, 109.6169662475586, 111.40321350097656, 114.26602935791016, 112.2437744140625, 109.07392120361328, 109.16975402832031, 107.95294189453125, 110.02708435058594, 110.11180114746094, 113.12081146240234, 109.57000732421875, 108.73204803466797, 108.83275604248047, 108.76665496826172, 108.86824035644531, 110.09841918945312, 110.09121704101562, 109.96764373779297, 107.59036254882812], [72.22057342529297, 74.84281921386719, 64.8193130493164, 67.5648422241211, 70.52532196044922, 67.50662231445312, 78.20569610595703, 65.0877914428711, 72.6133041381836, 89.07587432861328, 65.6043319702148

# 实验 Experiment

### 测试矩阵 Test matrix

In [7]:
m = [[0, 0, 0],
     [0, 0, 1],
     [1, 0, 1],
     [1, 0, 0],
     [1, 1, 0],
     [1, 1, 1],
     [1, 1, 1],
     [1, 1, 0]]

all_labels = AllLabelConfigurations(m)
print('total labels:',len(all_labels))

total labels: 10080


## Ising

### experimental function

In [8]:
import warnings
import pyphi

warnings.filterwarnings('ignore')

labels = ('A', 'B', 'C')

def IsingEnergyExp(exp_m):
    print('Computing Ising Energy...')
    ising_energy = [IsingEnergy(MapToKarnaughMap(alabel)) for alabel in exp_m]
    print('done.')
    
    print('Computing Phi...')
    phis = [getphi(alabel) for alabel in exp_m]
    print('done.')
    
    return ising_energy, phis

### experiment

In [11]:
# Simple test

testIsing, testPhi = IsingEnergyExp(all_labels[:2])

Computing concepts:   0%|          | 0/7 [00:00<?, ?it/s]

Computing Ising Energy...
done.
Computing Phi...


done.


In [9]:
list(range(0,10,2))

[0, 2, 4, 6, 8]

In [23]:
40 // 3

13

In [16]:
# Real experiment

import pandas as pd
import time

start = time.time()

total_test = 1000
isingEnergys, phis = IsingEnergyExp(all_labels[:total_test])

isingEnergyData = pd.DataFrame(isingEnergys)
isingEnergyData.to_csv('./data/isingEnergy_0_1000.csv')

PhiData = pd.DataFrame(phis)
PhiData.to_csv('./data/Phi_0_1000.csv')

end = time.time()

print('total time cost:', end-start, 's')

Computing concepts:   0%|          | 0/7 [00:00<?, ?it/s]

Computing Ising Energy...
done.
Computing Phi...


done.
total time cost: 1140.2095048427582 s


## Neuron Network

In [34]:
trainNet(m)

net(
  (fcn_v): Linear(in_features=3, out_features=3, bias=True)
  (fcn_v2h): Linear(in_features=3, out_features=4, bias=True)
  (fcn_h2v): Linear(in_features=4, out_features=3, bias=True)
)

In [39]:
energys = []

start = time.time()
# expect time: 3h
for i, graph in zip(range(100), all_labels):
    energys += [[NetEnergy(trainNet(graph.tolist())) for j in range(32)]]
    if i % 10 == 0:
        print(i/100,"%")
    
end = time.time()
print(energys)
print('time cost:', (end-start)/60, 'min')

0.0 %
0.1 %
0.2 %
0.3 %
0.4 %
0.5 %
0.6 %
0.7 %
0.8 %
0.9 %
[[125.3689193725586, 114.34842681884766, 77.66404724121094, 80.1391372680664, 100.38593292236328, 95.96160125732422, 79.7577896118164, 81.27345275878906, 96.10023498535156, 83.8078842163086, 88.44671630859375, 85.97489929199219, 81.16421508789062, 106.55809783935547, 118.70069122314453, 81.2342300415039, 87.16758728027344, 98.25985717773438, 82.06558990478516, 94.70956420898438, 91.53181457519531, 82.59296417236328, 126.27677917480469, 80.054931640625, 111.71385955810547, 85.78819274902344, 80.1546401977539, 106.10945892333984, 92.69000244140625, 80.84135437011719, 82.78980255126953, 83.82793426513672], [93.68472290039062, 81.88871765136719, 101.65992736816406, 85.98505401611328, 79.86389923095703, 82.20327758789062, 97.70317077636719, 135.9124298095703, 92.10265350341797, 85.17431640625, 108.1070785522461, 100.34458923339844, 96.89277648925781, 81.30494689941406, 77.02705383300781, 81.74012756347656, 80.44074249267578, 82.935

In [52]:
import pandas as pd

energyData = pd.DataFrame(energys)

energyData.to_csv('./data/energyData.csv')

In [53]:
first100Labels = []

for lb in all_labels[:100]:
    first100Labels += lb.tolist()
    
lbData = pd.DataFrame(first100Labels)
lbData.to_csv('./data/lbData.csv')

## $\Phi$

In [172]:
import pyphi

m = [[0, 0, 0],
     [0, 0, 1],
     [1, 0, 1],
     [1, 0, 0],
     [1, 1, 0],
     [1, 1, 1],
     [1, 1, 1],
     [1, 1, 0]]

cm = np.array([
     [0, 0, 1],
     [1, 0, 1],
     [1, 1, 0]
 ])

all_labels = AllLabelConfigurations(m)

print('total labels:', len(all_labels))

labels = ('A', 'B', 'C')

total labels: 10080


In [179]:
all_labels[903]

array([[1, 1, 0],
       [1, 0, 0],
       [1, 0, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 1, 1]])

In [10]:
def is_in(state, tpm):
    for astate in tpm:
        if state == astate.tolist():
            return True
    return False

def getphi(tpm):
    network = pyphi.Network(tpm, node_labels=labels)
    phis = []
    for i in range(2):
        for j in range(2):
            for k in range(2):
                state = (i, j, k)
                node_indices = (0, 1, 2)
                if is_in(list(state), tpm):
                    subsystem = pyphi.Subsystem(network, state, node_indices)
                    phis += [pyphi.compute.phi(subsystem)]
    return phis

In [19]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

phis = [getphi(alabel) for i, alabel in zip(range(len(all_labels)), all_labels)]

phisData=pd.DataFrame(phis)
phisData.to_csv('./data/phis.csv')

In [31]:
phis

[[0.1875, 0.1875, 2.3125, 0.687499, 0.1875, 0.1875],
 [0, 0, 0, 0, 0, 0],
 [0.168403, 0.150625, 0.08507, 0.067292, 0.08507, 0.067292],
 [0.28125, 0.28125, 1.215279, 0.708332, 0.197917, 0.197917],
 [0.09375, 0.09375, 0.4125, 0.06875, 0.06875],
 [0.09375, 0.088125, 0.063125, 0.06875, 0.063125],
 [0, 0, 0, 0, 0],
 [0.078125, 0.078125, 0.678125, 0.078125, 0.078125],
 [0, 0, 0, 0, 0],
 [0.18125, 0.209375, 0.888119, 0.428125, 0.18125, 0.209375],
 [0.07875, 0.096528, 0.063125, 0.080903, 0.063125, 0.080903],
 [0, 0, 0, 0, 0, 0],
 [0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625],
 [0.078125, 0.078125, 0.390625, 0.078125, 0.078125],
 [0.096528, 0.07875, 0.559219, 0.219375, 0.080903, 0.063125],
 [0.209375, 0.18125, 0.871874, 0.3375, 0.209375, 0.18125],
 [0.140625, 0.140625, 0.515625, 0.140625, 0.140625],
 [0, 0, 0.375, 0, 0],
 [0.0625, 0.0625, 0.635, 0.25, 0.0625, 0.0625],
 [0, 0, 0, 0, 0, 0],
 [0.078125, 0.078125, 1.459375, 0.078125, 0.078125],
 [0, 0, 0, 0, 0],
 [0.09375, 0.088125, 0.06875, 0.0

# Random

In [13]:
for arm in rm[:10]:
    #print(MapToKarnaughMap(arm))
    print(IsingEnergy(arm))

-24
-16
-16
-24
0
-16
-16
-16
-8
-24


In [14]:
import time

time.time()

1552528487.9991295